In [2]:
import pygame 
import numpy as np
import sys
from typing import Tuple

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def cartesian_to_pygame(window_height: int, coordinates: Tuple[float, float]) -> Tuple[float, float]:
    return coordinates[0], window_height - coordinates[1]

def homogeneous_coordinates(vertices):
    return np.concatenate((vertices, np.ones((vertices.shape[0], 1))), axis=1)

def homogeneous_to_cartesian(vertices: np.ndarray):
    return vertices[:, 0:-1] / vertices[:, -1][:, None]

In [21]:
def rotation():
    pass

def horizontal_translation(vertices, step):
    return vertices @ np.array([[1, 0, 0, 0],
                                [0, 1, 0, 0],
                                [0, 0, 1, 0],
                                [step, 0, 0, 1]])

def scale():
    pass

In [5]:
screen_width = 800
screen_height = 800
center = np.array([400, 400, 400])
half_width = 50
half_height = 50
half_depth = 50
camera_location = np.array([400, 400, 200])

In [13]:
x, y, z = np.meshgrid((center[0] + half_width, center[0] - half_width), 
                      (center[1] + half_height, center[1] - half_height), 
                      (center[2] + half_depth, center[2] - half_depth), copy=True)

vertices = np.column_stack((x.flatten(), y.flatten(), z.flatten()))
faces = np.array([[0, 4, 6, 2],
                  [1, 3, 7, 5],
                  [0, 2, 3, 1], 
                  [0, 1, 5, 4], 
                  [1, 3, 7, 5],
                  [2, 6, 7, 3]])
vertices, faces

(array([[450, 450, 450],
        [450, 450, 350],
        [350, 450, 450],
        [350, 450, 350],
        [450, 350, 450],
        [450, 350, 350],
        [350, 350, 450],
        [350, 350, 350]]),
 array([[0, 4, 6, 2],
        [1, 3, 7, 5],
        [0, 2, 3, 1],
        [0, 1, 5, 4],
        [1, 3, 7, 5],
        [2, 6, 7, 3]]))

In [14]:
homo_vertices = homogeneous_coordinates(vertices)
homo_vertices

array([[450., 450., 450.,   1.],
       [450., 450., 350.,   1.],
       [350., 450., 450.,   1.],
       [350., 450., 350.,   1.],
       [450., 350., 450.,   1.],
       [450., 350., 350.,   1.],
       [350., 350., 450.,   1.],
       [350., 350., 350.,   1.]])

In [42]:
homo_vertices = horizontal_translation(homo_vertices, 100)
homo_vertices

array([[660., 450., 450.,   1.],
       [660., 450., 350.,   1.],
       [560., 450., 450.,   1.],
       [560., 450., 350.,   1.],
       [660., 350., 450.,   1.],
       [660., 350., 350.,   1.],
       [560., 350., 450.,   1.],
       [560., 350., 350.,   1.]])

In [43]:
# create cuboid in 3d world space
# project cuboid onto 2d screen
# draw 2d projection

projected_vertices = homo_vertices @ np.array([[-camera_location[2], 0, 0], 
                     [0, -camera_location[2], 0], 
                     [camera_location[0], camera_location[1], 1], 
                     [0, 0, -camera_location[2]]])
projected_vertices

array([[ 48000.,  90000.,    250.],
       [  8000.,  50000.,    150.],
       [ 68000.,  90000.,    250.],
       [ 28000.,  50000.,    150.],
       [ 48000., 110000.,    250.],
       [  8000.,  70000.,    150.],
       [ 68000., 110000.,    250.],
       [ 28000.,  70000.,    150.]])

In [44]:
projected_vertices_cartesian = homogeneous_to_cartesian(projected_vertices)
projected_vertices_cartesian

array([[192.        , 360.        ],
       [ 53.33333333, 333.33333333],
       [272.        , 360.        ],
       [186.66666667, 333.33333333],
       [192.        , 440.        ],
       [ 53.33333333, 466.66666667],
       [272.        , 440.        ],
       [186.66666667, 466.66666667]])

In [ ]:
array([[272.        , 360.        ],
       [186.66666667, 333.33333333],
       [352.        , 360.        ],
       [320.        , 333.33333333],
       [272.        , 440.        ],
       [186.66666667, 466.66666667],
       [352.        , 440.        ],
       [320.        , 466.66666667]])

In [41]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))

screen.fill((255, 255, 255))

for face in faces:
    for i in range(face.shape[0]):
        pygame.draw.aaline(screen, (0, 0, 0), 
                           cartesian_to_pygame(screen_height, projected_vertices_cartesian[face[i]]), 
                           cartesian_to_pygame(screen_height, projected_vertices_cartesian[face[(i + 1) % face.shape[0]]]))

pygame.display.update()
    

In [34]:
pygame.quit()

SystemExit: 

/opt/miniconda3/envs/affine_transformations/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
